In [0]:
import os
import time
import argparse
import csv

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.backends.cudnn as cudnn

import numpy as np 
import pandas as pd 
import scipy.sparse as sp

import torch.utils.data as data

import pandas as pd
from collections import defaultdict

In [0]:
torch.cuda.current_device()
torch.cuda.is_available()

True

In [0]:
def load_all(test_num=50):
    """ We load all the three file here to save time in each epoch. """
    train_data = pd.read_csv("train.csv", 
        sep=',', header=None, names=['user', 'item'], 
        usecols=[0, 1], dtype={0: np.int32, 1: np.int32})

    user_num = train_data['user'].max() + 1
    item_num = train_data['item'].max() + 1

    users = np.unique(train_data.user)
    items = np.unique(train_data.item)

    train_data = train_data.values.tolist()

    # load ratings as a dok matrix
    train_mat = sp.dok_matrix((user_num, item_num), dtype=np.float32)
    for x in train_data:
        train_mat[x[0], x[1]] = 1.0

    # load the comunity
    comm_data = {}
    with open('top_comm.csv', newline='') as csvfile:
        read = csv.reader(csvfile, delimiter=',')
        for row in read:
            comm_data[int(row[0])] = [int(r) for r in row[1:]]
    

    test_data = []
    test_df = pd.read_csv("test.csv", header=None)
    for index, row in test_df.iterrows():
      # for negs in row[2:50]:
      test_data.append([int(row[0]), int(row[1])]) #, int(row[])])

    return train_data, test_data, user_num, item_num, train_mat, comm_data, users, items

In [0]:
############################## PREPARE DATASET ##########################
train_data, test_data, user_num ,item_num, train_mat, top_comm, users, items = load_all()


In [0]:
user_items = defaultdict(list)
item_users = defaultdict(list)
for user,item in train_data:
  user_items[user].append(item)
  item_users[item].append(user)
print(len(user_items), len(users), len(item_users), len(items))

4557 4557 2219 2219


In [0]:
import random 
bpr_dict = defaultdict(set)
def bpr(user):
    unseen_items = None
    if bpr_dict[user]==set():
        unseen_items = set(items).difference(user_items[user])
        bpr_dict[user] = unseen_items
    else:
        unseen_items = bpr_dict[user]
    return int(random.choice(list(unseen_items)))

In [0]:
# sbpr_friend_dict = defaultdict(list)  ## I already have this  

sbpr_negative_dict = defaultdict(list)
sbpr_friend_dict = defaultdict(list)
# if 'top_comm' in globals():
#   print("yes")
#   sbpr_friend_dict = top_comm
# else:
#   print("no")
  

user_saved_items = defaultdict(set)
def sbpr(user):
    if user not in top_comm:
        bpr_item = bpr(user)
        return (bpr_item,bpr_item)
    
    random_friend_item = None
    similar_interests = None
    
    # friends = list(g.adj[user])
    friends = top_comm[user]

    friend_items = set()
    similar_interests = 1
    if user_saved_items[user] == set():
      if sbpr_friend_dict[user] == list():
          for friend in friends:
              friend_items = friend_items.union(user_items[friend])
          friends_items = list(friend_items.difference(user_items[user]))
          sbpr_friend_dict[user] = friends_items
      else:
          friends_items = sbpr_friend_dict[user]
          # sbpr_friend_dict[user]= friends_items
    else:
      friends_items = user_saved_items[user]
    if friends_items:
      random_friend_item = int(random.choice(friends_items))
    else:
      bpr_item = bpr(user)
      return ([bpr_item,bpr_item])
    # if sbpr_weight[(user,random_friend_item)]!=0:
    #   similar_interests = sbpr_weight[(user,random_friend_item)]
    # else:
    #   for friend in friends:
    #     if random_friend_item in user_items[friend]:
    #         similar_interests+=1
    # random_friend_item = item_to_index[random_friend_item]
   
    if sbpr_negative_dict[user] == list(): # check this
        unseen_items = list(set(items).difference(friend_items.union(user_items[user])))
        sbpr_negative_dict[user] = unseen_items
    else:
        unseen_items = sbpr_negative_dict[user]
    random_item = int(random.choice(unseen_items))
    return (random_friend_item,int(random_item))

In [0]:
# for i in range(3000):
#   if i in sbpr_friend_dict.keys():
#     continue
#   else:
#     print(i)

type(sbpr(users[1])[0]), type(sbpr(users[1])[1])

(int, int)

In [0]:
## DataLoader

class BPRData(data.Dataset):
    def __init__(self, features, num_item, train_mat=None, num_ng=0, is_training=None):
        super(BPRData, self).__init__()
        """ Note that the labels are only useful when training, we thus 
            add them in the ng_sample() function.
        """
        self.features = features
        self.num_item = num_item
        self.train_mat = train_mat
        self.num_ng = num_ng
        self.is_training = is_training

    def ng_sample(self):
        # assert self.is_training, 'no need to sampling when testing'
        self.features_fill = []
        for x in self.features:
            u, i = x[0], x[1]
            for t in range(self.num_ng):
                j, k = sbpr(u)
                # j = np.random.randint(self.num_item)
                # while (u, j) in self.train_mat:
                #     j = np.random.randint(self.num_item)
                self.features_fill.append([u, i, j, k])

    def __len__(self):
        return self.num_ng * len(self.features) #if \
                #self.is_training else len(self.features)

    def __getitem__(self, idx):
        features = self.features_fill #if self.is_training else self.features
        # if idx==0:
        #   print(features[idx])
        user = features[idx][0]
        item_i = features[idx][1]
        item_j = features[idx][2] #this is the community/sbpr term
        item_k = features[idx][3]#.type(torch.LongTensor)
        if self.is_training:
          return user, item_i, item_j, item_k
        else:
          return user, item_i, item_k

In [0]:
import torch
import torch.nn as nn

# model
class BPR(nn.Module):
    def __init__(self, user_num, item_num, factor_num):
        super(BPR, self).__init__()
        """
        user_num: number of users;
        item_num: number of items;
        factor_num: number of predictive factors.
        """		
        self.embed_user = nn.Embedding(user_num, factor_num)
        self.embed_item = nn.Embedding(item_num, factor_num)

        nn.init.normal_(self.embed_user.weight, std=0.01)
        nn.init.normal_(self.embed_item.weight, std=0.01)

    def forward(self, user, item_i, item_j, item_k):
        # item_i = item_i.type(torch.LongTensor)
        # item_j = item_j.type(torch.LongTensor)
        # item_k = item_k.type(torch.LongTensor)
        item_i = item_i.cuda()
        item_j = item_j.cuda()
        item_k = item_k.cuda()


        user = self.embed_user(user)
        item_i = self.embed_item(item_i)
        item_j = self.embed_item(item_j)
        item_k = self.embed_item(item_k)

        prediction_i = (user * item_i).sum(dim=-1)
        prediction_j = (user * item_j).sum(dim=-1)
        prediction_k = (user * item_k).sum(dim=-1)
        return prediction_i, prediction_j, prediction_k


In [0]:
import numpy as np
import torch


def hit(gt_item, pred_items):
    if gt_item in pred_items:
        return 1
    return 0


def ndcg(gt_item, pred_items):
    if gt_item in pred_items:
        index = pred_items.index(gt_item)
        return np.reciprocal(np.log2(index+2))
    return 0


def metrics(model, test_loader):
    accuracy, auc = [], []
    tmp_auc = defaultdict(list)
    for user, item_i, item_j in test_loader:
        user = user.cuda()
        item_i = item_i.cuda()
        item_j = item_j.cuda()
        prediction_i, prediction_j, prediction_j = model(user.cuda(), item_i.cuda(), item_j.cuda(), item_j.cuda())
        # print(prediction_i)
        # print(prediction_j)
        count = 0
        pred = prediction_i > prediction_j
        pred = [int(p) for p in pred]
        accuracy.append(sum(pred)/len(pred))
    return np.mean(accuracy)

In [0]:

# construct the train and test datasets
train_dataset = BPRData(train_data, item_num, train_mat, 1, True)
test_dataset = BPRData(test_data, item_num, train_mat, 100, False)
train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

In [0]:
tmp = 0
train_loader.dataset.ng_sample()
test_loader.dataset.ng_sample()
for user, item_i, item_j in test_loader:
  # print(user,item_i,item_j)#, item_k)
  print(type(user),type(item_i),type(item_j)) #,type(item_k))
  print(user.type(), item_i.type(), item_j.type()) #, item_k.type())
  tmp += 1
  if tmp==1:
    break
# test_data

<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
torch.LongTensor torch.LongTensor torch.LongTensor


In [0]:
# top_comm[1]

In [0]:
########################### CREATE MODEL #################################
model = BPR(user_num, item_num, 8)
model.cuda()

BPR(
  (embed_user): Embedding(4798, 8)
  (embed_item): Embedding(2393, 8)
)

In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [0]:
########################### TRAINING #####################################
for epoch in range(100):
    model.train() 
    start_time = time.time()
    train_loader.dataset.ng_sample()
    

    for user, item_i, item_j, item_k in train_loader:
        # user = torch.tensor(user).to(torch.int64)
        # item_i = torch.tensor(item_i).to(torch.int64)
        # item_j = torch.tensor(item_j).to(torch.int64)
        # item_k = torch.tensor(item_k).to(torch.int64)

        user = user.cuda()
        item_i = item_i.cuda()
        item_j = item_j.cuda()
        item_k = item_k.cuda()

        model.zero_grad()
        prediction_i, prediction_j, prediction_k = model(user, item_i, item_j, item_k)
        loss = - (prediction_i - prediction_j).sigmoid().log().sum()
        loss -= (prediction_j - prediction_k).sigmoid().log().sum()
        loss.backward()
        optimizer.step()

    model.eval()
    test_loader.dataset.ng_sample()
    accuracy= metrics(model, test_loader)

    elapsed_time = time.time() - start_time
    print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
            time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
    print("accuracy : {}".format(accuracy))


The time elapse of epoch 000 is: 00: 00: 51
accuracy : 0.9184455202647777
The time elapse of epoch 001 is: 00: 00: 48
accuracy : 0.9274864635350134
The time elapse of epoch 002 is: 00: 00: 48
accuracy : 0.9304686592730229
The time elapse of epoch 003 is: 00: 00: 48
accuracy : 0.9302590799558704
The time elapse of epoch 004 is: 00: 00: 48
accuracy : 0.9317007316223435
The time elapse of epoch 005 is: 00: 00: 48
accuracy : 0.933131496051562
The time elapse of epoch 006 is: 00: 00: 48
accuracy : 0.9339952168737661
The time elapse of epoch 007 is: 00: 00: 48
accuracy : 0.9349587373708048
The time elapse of epoch 008 is: 00: 00: 48
accuracy : 0.9358170145743816
The time elapse of epoch 009 is: 00: 00: 48
accuracy : 0.9369148109975612
The time elapse of epoch 010 is: 00: 00: 48
accuracy : 0.9373648168040878
The time elapse of epoch 011 is: 00: 00: 48
accuracy : 0.9387719922192544
The time elapse of epoch 012 is: 00: 00: 48
accuracy : 0.9389235062710487
The time elapse of epoch 013 is: 00: 00

In [0]:
torch.save(model, "sbpr")

NameError: ignored

In [0]:
a = torch.tensor([1,2,3])
b = torch.tensor([0,1,1])
torch.sum(a>b)

tensor(3)